In [1]:
from sklearn.metrics import confusion_matrix
import numpy as np


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [9]:
y_true = ["a", "b", "c", "a", "b", "c", "a", "b", "c", "a", "b", "c", "a"]
y_pred = ["a", "b", "a", "c", "b", "c", "a", "b", "c", "a", "b", "c", "a"]

cm = confusion_matrix(y_true, y_pred, labels=["a", "b", "c"])
print(cm)

[[4 0 1]
 [0 4 0]
 [1 0 3]]


In [10]:
normalize_confusion_matrix = "all"

true_marginal = np.sum(cm, axis=1)[:,None]
pred_marginal = np.sum(cm, axis=0)[None,:]

num_observations = len(y_true)

true_rel_prop = true_marginal / num_observations
pred_rel_prop = pred_marginal / num_observations
all_rel_prop = 1
none_rel_prop = None

n_rows, n_cols = cm.shape

stds = np.zeros((n_rows, n_cols))

for i in range(n_rows):
    for j in range(n_cols):
        if normalize_confusion_matrix == 'true':
                obs_level_array = np.zeros(true_marginal[i, 0], dtype=int)
        elif normalize_confusion_matrix == 'pred':
            obs_level_array = np.zeros(pred_marginal[0, j], dtype=int)
        elif normalize_confusion_matrix == 'all':
            obs_level_array = np.zeros(num_observations, dtype=int)
        else:
            obs_level_array = np.zeros(num_observations, dtype=int)
        
        end_index = cm[i, j]
        obs_level_array[:end_index] = 1

        stds[i, j] = np.std(obs_level_array)

print(stds)




[[0.46153846 0.         0.26646936]
 [0.         0.46153846 0.        ]
 [0.26646936 0.         0.42132504]]


In [20]:
pred_rel_prop

array([[0.38461538, 0.30769231, 0.30769231]])

In [19]:
data = ['a', 'b', 'c']

if normalize_confusion_matrix == 'true':
    standard_errors = stds / np.sqrt(len(data) * true_rel_prop)
elif normalize_confusion_matrix == 'pred':
    standard_errors = stds / np.sqrt(len(data) * pred_rel_prop)
elif normalize_confusion_matrix == 'all':
    standard_errors = stds / np.sqrt(len(data) * all_rel_prop)
else:
    standard_errors = (stds / np.sqrt(len(data))) * len(data)

print(standard_errors)

[[0.26646936 0.         0.15384615]
 [0.         0.26646936 0.        ]
 [0.15384615 0.         0.24325213]]


In [22]:
test_denom = np.ones_like(true_rel_prop)
# multiplu first row by 1, second row by 2, third row by 3
test_denom = np.multiply(test_denom, np.arange(1, n_rows+1)[:, None])
test_denom = test_denom / np.sum(test_denom)
test_denom

array([[0.16666667],
       [0.33333333],
       [0.5       ]])

In [23]:
# make an array with the same shape as cm but with all 10s
test = np.ones((3, 3)) * 10
test / (test_denom * 10)

array([[6., 6., 6.],
       [3., 3., 3.],
       [2., 2., 2.]])

In [28]:
test_denom2 = np.ones_like(pred_rel_prop)
# multiplu first row by 1, second row by 2, third row by 3
test_denom2 = np.multiply(test_denom2, np.arange(1, n_cols+1)[None, :])
test_denom2 = test_denom2 / np.sum(test_denom2)
test_denom2

array([[0.16666667, 0.33333333, 0.5       ]])

In [29]:
test / (test_denom2 * 10)

array([[6., 3., 2.],
       [6., 3., 2.],
       [6., 3., 2.]])